## training Commands

### yolov9

In [ ]:
# chrono: 1

# gelan-c-default-parameters

# all 10 railSem19 classes
# default parameters
# 500 epochs

!python train.py --cfg models/detect/gelan-c.yaml --weights '' --name gelan-c-default-parameters --data /srv/cdl-eml/datasets/railsem19/rs19_split8500/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1

nach dem die resultate nicht wirklich gut sind habe ich die switches (unknown, left, right) rausgefiltert
daraus ist der subset vom railsem19 rausgekommen (onlySwitches)

In [ ]:
# chrono: 2

# gelan-c-default-parameters_onlySwitches
# only 3 switch classes (unknown, left, right)
# default parameters
# 500 epochs

!python train.py --cfg models/detect/gelan-c.yaml --weights '' --name gelan-c-default-parameters_onlySwitchs --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitches/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1

yolov9 bricht nach einer anzahl von epochen (100) das training ab, wenn es nicht mehr viel veränderung im validation loss gibt
um zu forcen dass alle 500 epochen ausgeführt werden muss man den parameter "--patience 0" setzen

In [ ]:
# chrono: 3

# gelan-c-default-parameters_onlySwitches2
# only 3 switch classes (unknown, left, right)
# default parameters
# 500 epochs --> geforced

!python train.py --cfg models/detect/gelan-c.yaml --weights '' --name gelan-c-default-parameters_onlySwitchs --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitches/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1 --patience 0

nachdem auch hier die ergebnisse nicht gerade gut sind habe ich das nächste experiment mit pretrained weights gemacht
dadurch dass der railSem19 datensatz relativ klein ist könnte das helfen (hat es auch)
die pretrained weights sind auf dem MS COCO datensatz trainiert worden

Im RailSem19 Paper wird angeführt dass sie ein densenet161 verwenden (sehr großes netz)
welches auf ImgNet pretrained ist und auf railSem19 gefine-tuned wurde (nur switch-left und switch-right klasse)
dabei wurden auch image crops expanded (30% in x und 125% in y) und danach labels die aus dem Bildrand ragen oder dimensionen haben die kleiner als 28 pxl sind ignoriert
--> nach 20 epochs trotzdem nur 67% accuracy!!!

weights für das yolov9 pretrained auf ImageNet habe ich nicht gefunden (deshalb COCO)

In [ ]:
# chrono: 4

# gelan-c-default-parameters_onlySwitches_pretrainedCOCO
# only 3 switch classes (unknown, left, right)
# default parameters
# 500 epochs --> geforced

!python train.py --cfg models/detect/gelan-c.yaml --weights ../weights/gelan-c.pt --name gelan-c-default-parameters_onlySwitches_pretrainedCOCO --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitches/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1 --patience 0

Aufgefallen ist ein kleiner drop in den letzten epochen vom train_loss
um sich das genauer anzuschauen habe ich die epochen von 500 auf 1000 erhöht (einfachster weg um sich das genauer anzuschauen)

In [ ]:
# chrono: 6

# gelan-c-default-parameters_onlySwitches_pretrainedCOCO2
# only 3 switch classes (unknown, left, right)
# default parameters
# 1000 epochs --> geforced

!python train.py --cfg models/detect/gelan-c.yaml --weights ../weights/gelan-c.pt --name gelan-c-default-parameters_onlySwitches_pretrainedCOCO --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitches/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 1000 --close-mosaic 15 --device 1 --patience 0

da die klasse switch-unknown nicht wirklich nützliche informationen enthält und diese information eigentlich 
unbrauchbar ist selbst wenn man diese richtig erkennen würde, habe ich frames mit switch-unknowns rausgefiltert

es ist ein subset von railsem19 entstanden in denen nur frames mit switch-left und switch-right behalten worden sind
es wurden somit nicht nur die labels switch-left und switch-right behalten, sondern sobald ein frame die klasse switch-unknown enthalen hat, ist der frame ignoriert worden (aktiv rausgefiltert)

dieser ansatz soll für die geringste verwirrung des models sorgen da die accuracy sehr niedrig ist und es einige false positives und false-negatives gibt

da das experiment chrono 6 schlechter abgeschnitten hat als chrono 4 nehme ich jetzt wieder nur 500 epchos 

In [ ]:
# chrono: 7
# gelan-c-default-parameters_onlySwitchesLeftRight_pretrainedCOCO
# only 2 switch classes (left, right)
# default parameters
# 500 epochs --> geforced

!python train.py --cfg models/detect/gelan-c.yaml --weights ../weights/gelan-c.pt --name gelan-c-default-parameters_onlySwitchesLeftRight_pretrainedCOCO --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitchesLeftRight/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1 --patience 0

Eine analyse aller confusion matizen hat ergeben dass bisher die onlySwitches_pretrainedCOCO (experiment chrono 4) am besten abgeschnitten hat

Begründung: vielleicht erklärung dadurch dass switch-unknown doch auf sehr vielen frames sind und bei chrono 7 einfach schon zuwenige daten da sind)
auf Grund dieser Annahme werden keiner weiteren subsets erstellt (eine idee wäre noch gewesen, dass man die frames löscht bei denen switches sehr klein und weit weg sind)

nun trainiere ich das gelan-e auf diese art und weise (default parameters, 500 epochs --> geforced, 3 klassen [switch-unknown, -left, -right], pretrained auf COCO)

In [ ]:
# chrono: 8

# gelan-e-default-parameters_onlySwitches_pretrainedCOCO
# only 3 switch classes (unknown, left, right)
# default parameters
# 500 epochs --> geforced
# ACHTUNG batch-size 32 ist zu groß !!! --> batch-size: 16

!python train.py --cfg models/detect/gelan-e.yaml --weights ../weights/gelan-e.pt --name gelan-e-default-parameters_onlySwitches_pretrainedCOCO --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitches/data.yaml --batch 16 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1 --patience 0

Erwartung: gelan-e wird am besten abschneiden ist aber größer als glan-c --> ist aber alles schlecht neuer ansatz muss her !!!!
gelan-e hat sogar schlechter abgeschnitten als gelan-c !!!!!!

Training data augmentation: flip left-right ist noch drinnen
Das verwirrt natürlich das modell

Das schalte ich nun aus in hyp.scratch-high.yaml (default: 0.5):
fliplr: 0.0 # 0.5  # image flip left-right (probability) # um keine flips drinnen zu haben

In [ ]:
# chrono: 10

# gelan-c-default-parameters_onlySwitches_pretrainedCOCO_noFlips
# only 3 switch classes (unknown, left, right)
# default parameters
# 500 epochs

!python train.py --cfg models/detect/gelan-c.yaml --weights ../weights/gelan-c.pt --name gelan-c-default-parameters_onlySwitches_pretrainedCOCO_noFlips --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitches/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1

- chrono 11 --> gelan-c nur mit left right   -> gemacht
- (vielleicht eins ohne pretrained)
- chrono 12 --> bestes experiment mit gelan-e -> gemacht
- chrono 13 --> bestes experiment mit yolov7

Erwartungen:
ohne Flips left-right wird das ergebnis besser aber noch immer nicht gut sein !!!

dann vielleicht noch verschiedene batch sizes ausprobieren (nur mit dem aller besten model)

nun da ich das genal-c auf alle drei switches traniert habe (ohne flip data augmentation), werde ich das experiment mit nur 2 klassen (switch-left, switch-right) wiederholen

In [ ]:
# chrono: 11

# gelan-c-default-parameters_onlySwitchesLeftRight_pretrainedCOCO_noFlips
# only 2 switch classes (left, right)
# default parameters
# 500 epochs

!python train.py --cfg models/detect/gelan-c.yaml --weights ../weights/gelan-c.pt --name gelan-c-default-parameters_onlySwitchesLeftRight_pretrainedCOCO_noFlips --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitchesLeftRight/data.yaml --batch 32 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1

bestes experiment (gelan-c-default-parameters_onlySwitchesLeftRight) wiederhole ich mit gelan-e

In [ ]:
# chrono: 12

# gelan-e-default-parameters_onlySwitchesLeftRight_pretrainedCOCO_noFlips
# only 2 switch classes (left, right)
# default parameters
# 500 epochs
# ACHTUNG batch-size 32 ist zu groß !!! --> batch-size: 16

!python train.py --cfg models/detect/gelan-e.yaml --weights ../weights/gelan-e.pt --name gelan-e-default-parameters_onlySwitchesLeftRight_pretrainedCOCO_noFlips --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitchesLeftRight/data.yaml --batch 16 --workers 8 --img 640 --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15 --device 1

### yolov7

nachdem das training mit den pre-trained weights von yolov9 am besten abgeschnitten hat habe ich das yolov7 auch mit pretrained weights trainiert (weights sind auf COCO trainiert)
training wurde nur mit den 3 klassen (unknown, left, right) durchgeführt

In [ ]:
# chrono: 5

# yolov7-default-parameters_onlySwitches_pretrainedCOCO
# only 3 switch classes (unknown, left, right)
# default parameters
# 500 epochs --> geforced

!python train.py --cfg cfg/training/yolov7.yaml --weights ../weights/yolov7_training.pt --name yolov7-default-parameters_onlySwitches_pretrainedCOCO --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitches/data.yaml --batch-size 32 --workers 8 --img 640 640 --hyp data/hyp.scratch.custom.yaml --epochs 500 --device 1

weitere experimente nicht furchgeführt da beim yolo v9 das Experiment "gelan-c-default-parameters_onlySwitches_pretrainedCOCO" (chrono 4) am besten abgeschnitten hat

Vollständigkeitshalber trainiere ich auch noch das yolov7 mit dem subset onlySwitchesLeftRight um zu sehen ob es auf das yolov7 einen positiven Einfluss hat.
Da es jedoch bereits beim yolov9 zu einem schlechteren ergebnis geführt hat (das liegt wahrscheinlich an den wenigen daten [nur noch 1240 frames]) ist die Erwartung hier, dass auch beim yolov7 die Ergebnisse negativ beeinflusst werden.

In [ ]:
# chrono: 9

# yolov7-default-parameters_onlySwitchesLeftRight_pretrainedCOCO
# only 2 switch classes (left, right)
# default parameters
# 500 epochs --> geforced

!python train.py --cfg cfg/training/yolov7.yaml --weights ../weights/yolov7_training.pt --name yolov7-default-parameters_onlySwitchesLeftRight_pretrainedCOCO --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitchesLeftRight/data.yaml --batch-size 32 --workers 8 --img 640 640 --hyp data/hyp.scratch.custom.yaml --epochs 500 --device 1

da die experimente chrono 12 und chrono 11 bis jt am besten abgeschnitten haben trainiere ich auch noch das yolov7 ohne der dataaugmentation left-right flips auf dem subdataset onlySwitchesLeftRight

In [ ]:
# chrono: 13

# yolov7-default-parameters_onlySwitchesLeftRight_pretrainedCOCO_noFlips
# only 2 switch classes (left, right)
# default parameters
# 500 epochs
# batch size auf 16 gesetzt weil der subdataset so klein ist

!python train.py --cfg cfg/training/yolov7.yaml --weights ../weights/yolov7_training.pt --name yolov7-default-parameters_onlySwitchesLeftRight_pretrainedCOCO_noFlips --data /srv/cdl-eml/datasets/railsem19/rs19_split8500_onlySwitchesLeftRight/data.yaml --batch-size 16 --workers 8 --img 640 640 --hyp data/hyp.scratch.custom.yaml --epochs 500 --device 1

### train-ego-path-detection

erstes trainning mit train-ego-path-detection:

In [ ]:
# chrono: 14
# global.yaml --> default (unverändert/wie in git repo)
# regression.yaml --> default außer epochs (epochs: 5)

# Name: good-dust-13

!python train.py regression resnet18 --device cuda:1

erstes training funktioniert!!!!!! :)

Mobilenetv3-small und -large als backbone eingebaut

In [ ]:
# chrono: 15
# global.yaml --> default (unverändert/wie in git repo)
# regression.yaml --> default außer epochs (epochs: 5)
# Backbone MobileNetV3-small (test run mit 5 epchos)

# Name: unique-galaxy-27

!python train.py regression mobilenet-small --device cuda:1

In [ ]:
# chrono: 16
# global.yaml --> default (unverändert/wie in git repo)
# regression.yaml --> default außer epochs (epochs: 5)
# Backbone MobileNetV3-large (test run mit 5 epchos)

# Name: magic-lake-28

!python train.py regression mobilenet-large --device cuda:1

nach dem die tests mit dem mobilenetv3-small und -large backbone durchgelaufen sind, starte ich ein erneutes (richtiges) training mit 400 epochen

In [ ]:
# chrono: 17
# global.yaml --> default (unverändert/wie in git repo)
# regression.yaml --> default (epochs: 400)
# Backbone MobileNetV3-small

# Name: sunny-haze-29

!python train.py regression mobilenet-small --device cuda:1

In [ ]:
# chrono: 18
# global.yaml --> default (unverändert/wie in git repo)
# regression.yaml --> default (epochs: 400)
# Backbone MobileNetV3-large

# Name: glowing-snowball-30

!python train.py regression mobilenet-large --device cuda:1

In [ ]:
# chrono: 19
# global.yaml --> default (unverändert/wie in git repo) --> input_shape: [3, 224, 224]
# regression.yaml --> default (epochs: 400)
# Backbone MobileNetV3-small

# Name: cosmic-forest-39

!python train.py regression mobilenet-small --device cuda:0

In [ ]:
# chrono: 20
# global.yaml --> default (unverändert/wie in git repo) --> input_shape: [3, 224, 224]
# regression.yaml --> default (epochs: 400)
# Backbone resnet18

# Name: dazzling-oath-40

!python train.py regression resnet18 --device cuda:0

In [ ]:
# chrono: 21
# global.yaml --> default (unverändert/wie in git repo) --> input shape: [3, 512, 512]
# regresseion.yaml --> default (epochs: 400)
# Backbone densenet121

# Name: hardy-wildflower-52

!python train.py regression densenet121 --device cuda:1